Дусаева Элина Эдуардовна 338859 Р32662

In [ ]:
!pip install pymorphy2 > None

In [ ]:
#установка библиотеки и подключение всех необходимых библиотек
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem.snowball import RussianStemmer
import nltk
import pandas as pd
import numpy as np
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#1.Использовать предобработанный корпус текстов из предыдущей лабы.
db = pd.read_csv('/content/result_laba2.csv', sep = ',')
db.head()

,Transcription,Emotion,Sentiment,Emotion digits,Sentiment digits,New_Transcription,New_Transcription_only_lemms
0,Давай ты успокоишься для начала. Ты можешь не ...,гнев,негативный сентимент,0,0,дава успоко нача моч вест ерунд эт специальн п...,давать успокоиться начало мочь вестись ерунда ...
1,"Подожди, подожди, успокойся пожалуйста. Подожд...",гнев,негативный сентимент,0,0,подожда подожда успоко пожалуйст подожда подож...,подождать подождать успокоиться пожалуйста под...
2,"Кристин, привет. Ты слышала там самолёт упал в...",страх,негативный сентимент,3,0,кристин привет слыша самолет упаст зеланд мы з...,кристин привет слышать самолёт упасть зеландия...
3,"Кристин, привет .. слушай .. Я боюсь лететь, т...",страх,негативный сентимент,3,0,кристин привет .. слуша .. боя летет разб само...,кристин привет .. слушать .. бояться лететь ра...
4,"Почему? Подожди... Ну подожди, ну как же, но м...",грусть,негативный сентимент,1,0,поч подожда ... подожда давн планирова эт кто-...,почему подождать ... подождать давно планирова...


#2.Векторизовать тексты при помощи тонального словаря - Dictionary_lemmatization_Rus.csv. Веса слов лежат в колонке Weight.


**Алгоритм векторизации текста при помощи тональных словарей**


1. Предобработать текст. Использовать лемматизацию.
2. Создать переменную vector типа list, куда будем записывать наш вектор.
3. Пройтись по каждому токену.
4. Если токен есть в тональном словаре, то добавить вес слова из тонального словаря в vector.
5. Если токена нет в тональном словаре, то в vector добавить 0.
6. В результате получим, что длина вектора совпадает с длиной текста.
7. Считаем статистические данные из полученного vector
> 1.   Кол-во положительных слов
> 2. Кол-во отрицаильных слов
> 3. Среднее значение сентимента всех слов
> 4. Максмально положительное значение вектора
> 5. Максимально отрицательное значение вектора
8. Записываем статистические данные из п.7 в переменную static_vector типа list. ***static_vector*** - вектор каждого текста, векторизиванного при помощи тональных словарей.

In [ ]:
#Загружаем тональный словарь
tonal_dictionary = pd.read_csv('/content/Dictionary_lemmatization_Rus.csv', sep = ',')
tonal_dictionary = tonal_dictionary[['Word', 'Weight']].set_index('Word').to_dict()['Weight']

In [ ]:
#функция, которая проходится по каждому слову и добавляет его вес в массив vector
def length_vector(row):
  token = word_tokenize(row['New_Transcription_only_lemms'])
  vector = []
  for i in token:
    if i in tonal_dictionary:
      vector.append(tonal_dictionary[i])
    else:
      vector.append(0)
  return vector

#функция обработки данных вектора и возвращение статистического вектора
def static_vector(vector):
  count_pos = 0
  count_neg = 0
  count_netral = 0
  average = 0
  max_pos = 0
  max_neg = 0
  mediana = 0

  for i in range(len(vector)):
    if vector[i] > 0:
      count_pos += 1
      if max_pos < vector[i]:
        max_pos = vector[i]
    elif vector[i] < 0:
      count_neg += 1
      if max_neg > vector[i]:
        max_neg = vector[i]
    else:
      count_netral += 1

  average = np.mean(vector)
  mediana = np.median(vector)
  static_vector = [count_pos, count_neg, average, max_pos, max_neg, count_netral,mediana]
  return static_vector

#функция для вывода статического вектора
def StaticVector(vector):
  vector = length_vector(vector)
  StatVec = static_vector(vector)
  return StatVec

#добавление в новую колонку статистического вектора
db['VECTOR'] = db.apply(length_vector, axis=1)
db['Static_vector'] = db.apply(StaticVector, axis=1)
db.head()

,Transcription,Emotion,Sentiment,Emotion digits,Sentiment digits,New_Transcription,New_Transcription_only_lemms,VECTOR,Static_vector
0,Давай ты успокоишься для начала. Ты можешь не ...,гнев,негативный сентимент,0,0,дава успоко нача моч вест ерунд эт специальн п...,давать успокоиться начало мочь вестись ерунда ...,"[0, 1.0, 0, 0, 0, -0.9166666666666664, 0, 0, -...","[2, 2, 0.0026041666666666748, 1.0, -1.0, 28, 0.0]"
1,"Подожди, подожди, успокойся пожалуйста. Подожд...",гнев,негативный сентимент,0,0,подожда подожда успоко пожалуйст подожда подож...,подождать подождать успокоиться пожалуйста под...,"[0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.0, 0, ...","[9, 3, 0.10252525252525252, 1.0, -1.0, 43, 0.0]"
2,"Кристин, привет. Ты слышала там самолёт упал в...",страх,негативный сентимент,3,0,кристин привет слыша самолет упаст зеланд мы з...,кристин привет слышать самолёт упасть зеландия...,"[0, 0, 0, 0, 0, 0.0, 0, 0, 0, 1.0, 0, 0, -1.0,...","[7, 8, -0.020149253731343283, 1.0, -1.0, 52, 0.0]"
3,"Кристин, привет .. слушай .. Я боюсь лететь, т...",страх,негативный сентимент,3,0,кристин привет .. слуша .. боя летет разб само...,кристин привет .. слушать .. бояться лететь ра...,"[0, 0, 0, 0, 0, -1.0, 0, 0, 0, 0, 0, -1.0, 0, ...","[5, 9, -0.035507246376811595, 1.0, -1.0, 55, 0.0]"
4,"Почему? Подожди... Ну подожди, ну как же, но м...",грусть,негативный сентимент,1,0,поч подожда ... подожда давн планирова эт кто-...,почему подождать ... подождать давно планирова...,"[0, 0, -1.0, 0, 0, 0, 0, 0, 0.0]","[0, 1, -0.1111111111111111, 0, -1.0, 8, 0.0]"


In [ ]:
#Сохранить все полученные вектора.
#Где каждый столбец это одно значение вектора, последний столбец – это сентимент вектора (текста), а строки – это вектор каждого текста == не смогла
db_tonal_dict = pd.DataFrame(db['VECTOR'])
db_tonal_dict['Static_vector'] = db['Static_vector']
db_tonal_dict['Sentiment digits'] = db['Sentiment digits']
db_tonal_dict.head()

,VECTOR,Static_vector,Sentiment digits
0,"[0, 1.0, 0, 0, 0, -0.9166666666666664, 0, 0, -...","[2, 2, 0.0026041666666666748, 1.0, -1.0, 28, 0.0]",0
1,"[0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0.0, 0, ...","[9, 3, 0.10252525252525252, 1.0, -1.0, 43, 0.0]",0
2,"[0, 0, 0, 0, 0, 0.0, 0, 0, 0, 1.0, 0, 0, -1.0,...","[7, 8, -0.020149253731343283, 1.0, -1.0, 52, 0.0]",0
3,"[0, 0, 0, 0, 0, -1.0, 0, 0, 0, 0, 0, -1.0, 0, ...","[5, 9, -0.035507246376811595, 1.0, -1.0, 55, 0.0]",0
4,"[0, 0, -1.0, 0, 0, 0, 0, 0, 0.0]","[0, 1, -0.1111111111111111, 0, -1.0, 8, 0.0]",0


#3.Векторизовать тексты с помощью BoW.

In [ ]:
#записываем метод векторизации в переменную vectorizer_BoW
vectorizer_BoW = CountVectorizer()

In [ ]:
#наш корпус текстов задаем черех столбец лемантизированных данных
corpus = db['New_Transcription_only_lemms']

In [ ]:
BoW = vectorizer_BoW.fit_transform(corpus).toarray()

#Сохранить все полученные вектора.
#Где каждый столбец это одно значение вектора, последний столбец – это сентимент вектора (текста), а строки – это вектор каждого текста
db_BoW = pd.DataFrame(BoW, columns = vectorizer_BoW.get_feature_names_out())
db_BoW['Sentiment digits'] = db['Sentiment digits']
db_BoW

,10,11,12,15,17,18,20,23,30,36,...,яковливич,ялта,январь,яндекс,ярмарка,ясно,яхта,яхточка,ёлка,Sentiment digits
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
531,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


#4.Векторизовать тексты с помощью TF-IDF.

In [ ]:
#записываем метод векторизации в переменную vectorizer_TF
vectorizer_TF = TfidfVectorizer()

In [ ]:
TF = vectorizer_TF.fit_transform(corpus).toarray()

#Сохранить все полученные вектора.
#Где каждый столбец это одно значение вектора, последний столбец – это сентимент вектора (текста), а строки – это вектор каждого текста
db_TF = pd.DataFrame(TF, columns = vectorizer_TF.get_feature_names_out())
db_TF['Sentiment digits'] = db['Sentiment digits']
db_TF

,10,11,12,15,17,18,20,23,30,36,...,яковливич,ялта,январь,яндекс,ярмарка,ясно,яхта,яхточка,ёлка,Sentiment digits
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,2
531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.101637,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,2
532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,2
533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,2


#5.Сохранить полученные результаты

In [ ]:
db_tonal_dict.to_csv('/content/db_tonal_dict.csv', index=False)
db_BoW.to_csv('/content/BoW.csv', index=False)
db_TF.to_csv('/content/TF-IDF.csv', index=False)